---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [180]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [181]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [182]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_csv("university_towns.txt" , sep=";", header=None)
    data =[]
    stateName = ""
    for i in range(df.shape[0]):
        val = str(df.iloc[i])
        if "edit" in val:
            stateName = val.split("[")[0][5:]
        else:
            region = val.split("(")[0][1:].strip().split("\n")[0]
            if region[len(region)-1] == ':' or region[len(region)-1] == ',':
                region = region[:len(region)-1].strip()
            elif region[len(region)-1] == '.':
                region = region[:len(region)-21].strip()
            data.append((stateName,region))
    df_cleaned = pd.DataFrame(data, columns = ["State", "RegionName"])
    return df_cleaned

In [183]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    cols = ["Quarterly(Seasonally adjusted annual rates)","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]
    df = df.iloc[219:,4:7]
    df.columns = cols
    df = df.loc[:,["Quarterly(Seasonally adjusted annual rates)","GDP in billions of chained 2009 dollars"]]
    for i in range(df.shape[0]-2):
        GDPValcurrent = df.iloc[i,1]
        GDPValnext1 = df.iloc[i+1,1]
        GDPValnext2 = df.iloc[i+2,1]
        if(GDPValnext2 < GDPValnext1 and GDPValnext1 < GDPValcurrent):
            index_val = df[df['GDP in billions of chained 2009 dollars']==GDPValnext1].index.values
            output = df.loc[index_val]["Quarterly(Seasonally adjusted annual rates)"].values[0]
            break
    return output

In [184]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    cols = ["Quarterly(Seasonally adjusted annual rates)","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]
    df = df.iloc[252:,4:7]
    df.columns = cols
    df = df.loc[:,["Quarterly(Seasonally adjusted annual rates)","GDP in billions of chained 2009 dollars"]]
    for i in range(df.shape[0]-2):
        GDPValcurrent = df.iloc[i,1]
        GDPValnext1 = df.iloc[i+1,1]
        GDPValnext2 = df.iloc[i+2,1]
        if(GDPValnext2 > GDPValnext1 and GDPValnext1 > GDPValcurrent):
            index_val = df[df['GDP in billions of chained 2009 dollars']==GDPValnext2].index.values
            output = df.loc[index_val]["Quarterly(Seasonally adjusted annual rates)"].values[0]
            break
    return output

In [185]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    cols = ["Quarterly(Seasonally adjusted annual rates)","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]
    recession_range = (252,258)
    df = df.iloc[252:258,4:7]
    df.columns = cols
    df = df.loc[:,["Quarterly(Seasonally adjusted annual rates)","GDP in billions of chained 2009 dollars"]]
    for i in range(df.shape[0]-2):
        GDPValcurrent = df.iloc[i,1]
        GDPValnext1 = df.iloc[i+1,1]
        GDPValnext2 = df.iloc[i+2,1]
        if(GDPValnext2 > GDPValnext1 and GDPValnext1 < GDPValcurrent):
            index_val = df[df['GDP in billions of chained 2009 dollars']==GDPValnext1].index.values
            output = df.loc[index_val]["Quarterly(Seasonally adjusted annual rates)"].values[0]
    return output

In [186]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    cols = ["State","RegionName"]
    for i in df.columns[51:].tolist():
        cols.append(i)
    df = df.loc[:,cols]
    quarterColumns = []
    i = 2
    while i < len(cols):
        for j in range(3):
            quarterColumns.append(cols[i][0:4]+"q1")
        for j in range(3):
            quarterColumns.append(cols[i][0:4]+"q2")
        for j in range(3):
            quarterColumns.append(cols[i][0:4]+"q3")
        for j in range(3):
            quarterColumns.append(cols[i][0:4]+"q4")
        i += 12
    region = ["State","RegionName"]
    quarterColumns = region + quarterColumns[:len(quarterColumns) - 4]
    df.columns = quarterColumns
    df1 = df.T.iloc[2:,:]
    df1 = df1.reset_index().replace(np.NaN,0)
    df1 = df1.groupby('index').mean().T
    df = pd.concat([df.iloc[:,0:2], df1], axis=1)
    df['State'] = [states[state] for state in df['State']]
    df = df.set_index(region).replace(0,np.NaN)
    return df

In [187]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    universityTowns = get_list_of_university_towns()
    rec_bottom = get_recession_bottom()
    rec_start = get_recession_start()
    df_house = convert_housing_data_to_quarters()
    df_start = df_house.columns[df_house.columns.get_loc(rec_start) - 1]
    df_house['ratio'] =  df_house[df_start] - df_house[rec_bottom]
    df_house = df_house[[rec_bottom, df_start, 'ratio']]
    df_house = df_house.reset_index()
    unitowns_housedata = pd.merge(df_house,universityTowns,how='inner',on=['State','RegionName'])
    unitowns_housedata['uni'] = True
    df_house2 = pd.merge(df_house, unitowns_housedata, how='outer', on=['State','RegionName',rec_bottom, df_start, 'ratio'])
    df_house2['uni'] = df_house2['uni'].fillna(False)
    ut = df_house2[df_house2['uni'] == True]
    nut = df_house2[df_house2['uni'] == False]
    t,p = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    different = True if p<0.01 else False
    better = "university town" if ut['ratio'].mean() < nut['ratio'].mean() else "non-university town"
    return(different, p, better)
run_ttest()

(True, 0.0021158846906012252, 'university town')